# Check GPU status

Make surre to use : GPU runtime mode (Runtime->Change Runtime type -> python3 + GPU
)



In [1]:
# Check nvidia and nvcc cuda compiler

!nvidia-smi
!/usr/local/cuda/bin/nvcc --version

Tue Feb 16 02:24:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Mount Goolge Drive

In [2]:
# link to google drive

from google.colab import drive
drive.mount('/content/gdrive/')


Mounted at /content/gdrive/


In [3]:
#check that Gdrive is mounted

!ls '/content/gdrive/My Drive/Colab Notebooks'

'16_nlp_with_rnns_and_attention (1).ipynb'
 16_nlp_with_rnns_and_attention.ipynb
 Bert-GPU.ipynb
 Chap1_Introduction_PyTorch.ipynb
 Chap3_NN_Components.ipynb
 Chap4_MLP_CNN.ipynb
 Chap5.ipynb
'Chap8_Encoder–decoder (1).ipynb'
 Chap8_Encoder–decoder.ipynb
'ColabEnv (1).ipynb'
 ColabEnv.ipynb
'Copy of Welcome To Colaboratory'
'Copy of Welcome To Colaboratory (1)'
'Copy of Welcome To Colaboratory (2)'
 CUDA_GPU_server.ipynb
 data
 model_storage
 NLP-LOTClass.ipynb
 pyTorchApexipynb.ipynb
 Transformers.ipynb
 Untitled
 Untitled0.ipynb
 Untitled1.ipynb


#Setup SSH port forwarding

In [4]:
#1 - setup ssh/user 


#Generate a random root password
import random, string
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(30))


#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null

#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config

print("username: root")
print("password: ", password)

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')


Creating config file /etc/ssh/sshd_config with new version
Creating SSH2 RSA key; this may take some time ...
2048 SHA256:WV1TD+gq1z6rEVvVVrDOzxnqXxkZ4gXRlCn0R54z1rE root@3643e2525dab (RSA)
Creating SSH2 ECDSA key; this may take some time ...
256 SHA256:6cUurFkh4QN4fwYZeohxmCIBTQFhza/5wVG000JpSK8 root@3643e2525dab (ECDSA)
Creating SSH2 ED25519 key; this may take some time ...
256 SHA256:obh2RWcjBZS3LgfEjeh681m+n49fossrHbZLElgvFg0 root@3643e2525dab (ED25519)
Created symlink /etc/systemd/system/sshd.service → /lib/systemd/system/ssh.service.
Created symlink /etc/systemd/system/multi-user.target.wants/ssh.service → /lib/systemd/system/ssh.service.
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
username: root
password:  jaf8Hu0jZXntiED8ltcZbSUcXNQWfF


In [5]:
# 2 - Download Ngrok

! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip

In [6]:
# 3 - setup Ngrok - authtoken

#Ask token
print("Get your authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()

#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')

Get your authtoken from https://dashboard.ngrok.com/auth
··········


Congratulations, you are ready to go. 
On Ngrok interface https://dashboard.ngrok.com/status you'll find the tcp address and the port 

connect using the following : 

```
ssh root@0.tcp.ngrok.io -p [ngrok_port]

> then enter the password generated previously

ssh root@2.tcp.ngrok.io -p 12024
```

In [ ]:
# When done, kill Ngrok

!kill $(ps aux | grep './ngrok' | awk '{print $2}')

In [7]:
# Mount Google Drive and make some folders for vscode
from google.colab import drive
drive.mount('/googledrive')
! mkdir -p /googledrive/My\ Drive/colabdrive
! mkdir -p /googledrive/My\ Drive/colabdrive/root/.local/share/code-server
! ln -s /googledrive/My\ Drive/colabdrive /
! ln -s /googledrive/My\ Drive/colabdrive/root/.local/share/code-server /root/.local/share/

Mounted at /googledrive


In [8]:
! curl -fsSL https://code-server.dev/install.sh | sh > /dev/null
! code-server --bind-addr 127.0.0.1:9999 --auth none &


######################################################################## 100.0%
[2021-02-16T03:29:01.877Z] info  Wrote default config file to ~/.config/code-server/config.yaml
[2021-02-16T03:29:02.346Z] info  code-server 3.9.0 fc6d123da59a4e5a675ac8e080f66e032ba01a1b
[2021-02-16T03:29:02.348Z] info  Using user-data-dir ~/.local/share/code-server
[2021-02-16T03:29:02.368Z] info  Using config file ~/.config/code-server/config.yaml
[2021-02-16T03:29:02.368Z] info  HTTP server listening on http://127.0.0.1:9999 
[2021-02-16T03:29:02.369Z] info    - Authentication is disabled 
[2021-02-16T03:29:02.369Z] info    - Not serving HTTPS 


In [ ]:
ssh -L 9999:localhost:9999 root@2.tcp.ngrok.io -p 12024

In [ ]:
https://towardsdatascience.com/colab-free-gpu-ssh-visual-studio-code-server-36fe1d3c5243